In [23]:
import torch
import numpy as np

print(f"Is GPU available? {torch.cuda.is_available()}")
print(f"PyTorch version: {torch.__version__}")

Is GPU available? True
PyTorch version: 2.8.0+cu126


### Tensors

In [ ]:
tensor0d = torch.tensor(1)
tensor1d = torch.tensor([1, 2, 3])
tensor2d = torch.tensor([[1, 2],
                         [3, 4]])
tensor3d = torch.tensor([[[1, 2], [3, 4]],
                         [[5, 6], [7, 8]]])

nptensor = np.array([[1, 2], [3, 4]])
tensorfromnumpy = torch.from_numpy(nptensor)

print("Number of tensors' dimensions:")
print(tensor0d.ndim)
print(tensor1d.ndim)
print(tensor2d.ndim)
print(tensor3d.ndim)

RuntimeError: Numpy is not available

In [22]:
floatvec = torch.tensor([1.0, 2.0, 3.0])
print("Tensors datatypes:")
print(floatvec.dtype)
print(floatvec.to(torch.int64).dtype)

Tensors datatypes:
torch.float32
torch.int64
